In [1994]:
import pandas as pd
import numpy as np
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
import string

In [1995]:
# Create a DataFrame
df = pd.DataFrame()

In [1996]:
def textToIntUTF(text):
    # Convert the text to bytes using UTF-8 encoding
    byte_representation = text.encode('utf-8')
    # Convert the bytes to integer
    integer_representation = int.from_bytes(
        byte_representation, byteorder='big')
    return integer_representation


def normalize(num):
    min = 35322350018592
    max = 139081753198206
    return (num - min)/(max-min)


def denormalize(num):
    min = 35322350018592
    max = 139081753198206
    return num*(max-min) + min


def intToTextUTF(num):
    # Convert the integer to bytes using UTF-8 encoding
    byte_representation = num.to_bytes(
        (num.bit_length() + 7) // 8, byteorder='big')
    # Convert the bytes to string using UTF-8 encoding
    text = byte_representation.decode('utf-8')
    return text


print(textToIntUTF('~~~'))  # 139081753198206
print(textToIntUTF('   '))  # 35322350018592

text = "odha#@"
print(normalize(textToIntUTF(text)))
res = normalize(textToIntUTF(text))
print(textToIntUTF(text))

res = denormalize(res)

print(intToTextUTF(int(res)))

# 2
# 1.999999
# Higher Precision Means more iterations with root finding algorithm

8289918
2105376
0.8399690622714339
122477038609216
odha#@


In [1997]:
samples = 1000
rand_mat_samples = 100
start = 0
end = 100
odd_start = 3
odd_end = 14  # python ignores the last number
text_size = 4

range_extend = 3000
max_iter = 25

tol = 10**(-2)

# max_val = 8289918
# min_val = 2105376

max_val = 2122219134
min_val = 538976288

In [1998]:
def x_random_numbers():
    x1 = np.random.randint(start, end)
    x2 = np.random.randint(start, end)
    while x1 == x2:
        x2 = np.random.randint(start, end)
    return min(x1, x2), max(x1, x2)


x = []
# Generate random numbers and add them to the DataFrame
for i in range(samples):
    x1, x2 = x_random_numbers()
    x.append([x1, x2])

df['x'] = x

# Display the first few rows of the DataFrame
df.head()

,x
0,"[12, 32]"
1,"[60, 97]"
2,"[43, 65]"
3,"[38, 81]"
4,"[59, 85]"


In [1999]:
def generate_numbers(df, start=start+1, end=end, samples=samples):
    numbers = np.random.randint(start, end, samples)
    df['y'] = [[-num, num] for num in numbers]


generate_numbers(df)

In [2000]:
def random_state(df, start=start, end=end, samples=None):
    samples = len(df) if samples is None else samples
    df['random_state'] = np.random.randint(start, end, samples)


random_state(df)
df.head()

,x,y,random_state
0,"[12, 32]","[-10, 10]",75
1,"[60, 97]","[-84, 84]",70
2,"[43, 65]","[-85, 85]",69
3,"[38, 81]","[-55, 55]",15
4,"[59, 85]","[-35, 35]",71


In [2001]:
odd_numbers = np.random.choice(range(odd_start, odd_end, 2), samples)
df['sections'] = odd_numbers

In [2002]:
random_mat = pd.read_csv('random_matrix.csv')
random_mat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1000094,1000021,1000000,1000022,1000090,1000039,1000002,1000047,1000046,1000083,1000014,1000059,1000096,1000069
1,1000066,1000090,1000045,1000079,1000077,1000044,1000088,1000045,1000006,1000069,1000059,1000067,1000085,1000010
2,1000091,1000042,1000021,1000099,1000069,1000078,1000024,1000004,1000080,1000027,1000066,1000041,1000067,1000062
3,1000051,1000050,1000017,1000055,1000023,1000049,1000066,1000037,1000071,1000005,1000003,1000033,1000057,1000075
4,1000036,1000074,1000072,1000056,1000098,1000005,1000017,1000083,1000018,1000054,1000081,1000044,1000069,1000024


In [2003]:
def add_random_mat(df, random_mat):
    def get_random_mat_row(row):
        index = row['random_state'] % (rand_mat_samples)
        return random_mat.iloc[index].values

    df['random_mat_row'] = df.apply(get_random_mat_row, axis=1)
    return df


add_random_mat(df, random_mat)

,x,y,random_state,sections,random_mat_row
0,"[12, 32]","[-10, 10]",75,13,"[1000003, 1000016, 1000094, 1000023, 1000070, ..."
1,"[60, 97]","[-84, 84]",70,9,"[1000005, 1000013, 1000070, 1000032, 1000091, ..."
2,"[43, 65]","[-85, 85]",69,9,"[1000023, 1000047, 1000078, 1000065, 1000049, ..."
3,"[38, 81]","[-55, 55]",15,5,"[1000000, 1000055, 1000016, 1000017, 1000018, ..."
4,"[59, 85]","[-35, 35]",71,3,"[1000096, 1000021, 1000036, 1000062, 1000065, ..."
...,...,...,...,...,...
995,"[63, 84]","[-34, 34]",38,11,"[1000079, 1000008, 1000052, 1000049, 1000042, ..."
996,"[84, 89]","[-44, 44]",16,5,"[1000032, 1000086, 1000086, 1000075, 1000007, ..."
997,"[28, 52]","[-58, 58]",77,13,"[1000079, 1000042, 1000097, 1000007, 1000053, ..."
998,"[2, 35]","[-18, 18]",31,11,"[1000072, 1000019, 1000069, 1000055, 1000002, ..."


In [2004]:
# Function to generate evenly spaced numbers
def generate_points_y(row):
    return list(np.linspace(row['y'][0], row['y'][1], row['sections']+1))


def generate_points_x(row):
    return list(np.linspace(row['x'][0], row['x'][1], row['sections']+1))


# Apply the function to each row
df['x_points'] = df.apply(generate_points_x, axis=1)
df['y_points'] = df.apply(generate_points_y, axis=1)

df.head()

,x,y,random_state,sections,random_mat_row,x_points,y_points
0,"[12, 32]","[-10, 10]",75,13,"[1000003, 1000016, 1000094, 1000023, 1000070, ...","[12.0, 13.538461538461538, 15.076923076923077,...","[-10.0, -8.461538461538462, -6.923076923076923..."
1,"[60, 97]","[-84, 84]",70,9,"[1000005, 1000013, 1000070, 1000032, 1000091, ...","[60.0, 64.11111111111111, 68.22222222222223, 7...","[-84.0, -65.33333333333333, -46.66666666666666..."
2,"[43, 65]","[-85, 85]",69,9,"[1000023, 1000047, 1000078, 1000065, 1000049, ...","[43.0, 45.44444444444444, 47.888888888888886, ...","[-85.0, -66.11111111111111, -47.22222222222222..."
3,"[38, 81]","[-55, 55]",15,5,"[1000000, 1000055, 1000016, 1000017, 1000018, ...","[38.0, 46.6, 55.2, 63.8, 72.4, 81.0]","[-55.0, -33.0, -11.0, 11.0, 33.0, 55.0]"
4,"[59, 85]","[-35, 35]",71,3,"[1000096, 1000021, 1000036, 1000062, 1000065, ...","[59.0, 67.66666666666667, 76.33333333333333, 8...","[-35.0, -11.666666666666668, 11.66666666666666..."


In [2005]:
df['rand_vals'] = df.apply(
    lambda row: row['random_mat_row'][:row['sections']+1], axis=1)

In [2006]:
df['points'] = df.apply(lambda row: list(
    zip(row['x_points'], row['y_points'])), axis=1)

In [2007]:
def update_points(row):
    new_points = []
    for i, (x, y) in enumerate(row['points']):
        if i % 2 == 0:  # subtract for even index
            new_y = y - row['rand_vals'][i]
        else:  # add for odd index
            new_y = y + row['rand_vals'][i]
        new_points.append((x, new_y))
    return new_points


df['poly_points'] = df.apply(update_points, axis=1)

In [2008]:
def interpolate_points(row):
    x, y = zip(*row['poly_points'])
    poly = lagrange(x, y)
    return Polynomial(poly).coef.tolist()


df['polynomial'] = df.apply(interpolate_points, axis=1)

In [2009]:
def generate_random_text(length):
    chars = np.array(list(string.ascii_letters + string.digits))
    text = ''.join(np.random.choice(chars) for _ in range(length))
    return text


df['rand_text'] = df.apply(lambda _: generate_random_text(text_size), axis=1)

In [2010]:
# Function to convert text to its UTF-8 integer representation
def text_to_int(text):
    return int.from_bytes(text.encode('utf-8'), 'big')


# Apply the function to the 'rand_text' column and create a new column 'text_int'
df['text_int'] = df['rand_text'].apply(text_to_int)

In [2011]:
# Normalize the 'text_int' column
df['text_normalized'] = (df['text_int'] - min_val) / (max_val - min_val)

In [2012]:
def subtract_normalized(row):
    polynomial = list(row['polynomial'])
    normalized_value = float(row['text_normalized'])
    polynomial[-1] -= normalized_value
    return polynomial


df['polynomial_text_normalized'] = df.apply(subtract_normalized, axis=1)

In [2013]:
def subtract_normalized(row):
    polynomial = list(row['polynomial'])
    text_int_value = float(row['text_int'])
    polynomial[-1] -= text_int_value
    return polynomial


df['polynomial_text_int'] = df.apply(subtract_normalized, axis=1)

In [2014]:
# # Define a function to subtract the normalized value from the constant term in the polynomial
# def subtract_normalized(row):
#     polynomial = row['polynomial'].copy()
#     normalized_value = row['text_int_normalized']
#     polynomial[-1] -= normalized_value
#     return polynomial


# # Apply the function to each row and store the results in a new column
# df['polynomial_text'] = df.apply(subtract_normalized, axis=1)

In [2015]:
# Subtract the last value of 'polynomial' from the last value of 'polynomial_text' and store the result in 'poly_text_representation'
df['poly_normalized_text_representation'] = df['polynomial'].apply(
    lambda x: x[-1]) - df['polynomial_text_normalized'].apply(lambda x: x[-1])

In [2016]:
# Subtract the last value of 'polynomial' from the last value of 'polynomial_text' and store the result in 'poly_text_representation'
df['poly_int_text_representation'] = df['polynomial'].apply(
    lambda x: x[-1]) - df['polynomial_text_int'].apply(lambda x: x[-1])

In [2017]:
# df[['polynomial', 'polynomial_text']].to_csv('polynomial.csv', index=False)

In [2018]:
# save the first value in the array polynomial in polynomial column in a separate column
df['first_polynomial_coeff'] = df['polynomial'].copy().apply(lambda x: x[0])

In [2019]:
# Get a Boolean Series where each element is True if the
# corresponding value in the 'first_polynomial_value' column
# is greater than 1
greater_than_zero = df['first_polynomial_coeff'].gt(1)

# Count the number of True values in the Series
count = greater_than_zero.sum()

print(count)

520


In [2020]:
df = df.loc[df['first_polynomial_coeff'] > 1]
df = df.loc[df['poly_int_text_representation'] != 0.0]

In [2021]:
# import pandas as pd
# df = pd.read_csv("data.csv")

In [2022]:
# f is a polynomial with the coeffs in polynomial_text_int
# a is the first value in x interval
# b is the first value in x interval+300


def blendBF(f, a, b, tol=tol, max_iter=25):
    # print(f)
    # Initialize the variables
    n = 0
    a1 = a
    a2 = a
    b1 = b
    b2 = b
    # print(f"working on {a} to {b} and {f}")
    while True:
        # print(n)
        # Increment the iteration counter
        n += 1

        # Check if the number of iterations exceeds the maximum number of iterations
        if n > max_iter:
            return 'iter'
        # Evaluate the function at the endpoints
        fa = f(a)
        fb = f(b)

        # Compute the midpoint and the false position point
        xB = (a + b) / 2
        fxB = f(xB)

        if fb == fa:
            return 'div'
        else:
            xF = a - (fa * (b - a)) / (fb - fa)
            xF = a - (fa * (b - a)) / (fb - fa)
            fxF = f(xF)

        # Choose the one with the smaller absolute value as the root approximation
        if abs(fxB) < abs(fxF):
            x = xB
            fx = fxB
        else:
            x = xF
            fx = fxF

        # Check if the absolute value of fx is less than or equal to the tolerance
        if abs(fx) <= tol:
            # Return the output
            # n is the number of iterations
            # x is the root approximation
            # fx is the function value
            # a is the left endpoint
            # b is the right endpoint
            return x

        # Update the interval by applying the bisection and false position methods
        if fa * fxB < 0:
            b1 = xB
        else:
            a1 = xB

        if fa * fxF < 0:
            b2 = xF
        else:
            a2 = xF

        # Set a to the maximum of a1 and a2 and b to the minimum of b1 and b2
        a = max(a1, a2)
        b = min(b1, b2)

In [2023]:
# # Define the bisection function
# def bisection(f, a, b, tol=tol, max_iter=max_iter):
#     # Check if a and b bracket a root
#     if f(a) * f(b) > 0:
#         return 'No root found in the interval', 'No root found in the interval'
#     # Initialize x as the midpoint of a and b
#     x = (a + b) / 2
#     # Initialize an iteration counter
#     i = 1
#     n = 0
#     # Repeat until the interval is smaller than the tolerance
#     while abs(b - a) > tol:
#         n += 1
#         if n > max_iter:
#             return 'iter'
#         # If x is a root, return it
#         if f(x) == 0:
#             return x, f(x), i, a, b
#         # If f(a) and f(x) have opposite signs, set b = x
#         elif f(a) * f(x) < 0:
#             b = x
#         # If f(b) and f(x) have opposite signs, set a = x
#         else:
#             a = x
#         # Update x as the new midpoint of a and b
#         x = (a + b) / 2
#         i += 1
#     return x

In [2024]:
# def falsePosition(f, x0, x1, tol=tol, max_iter=max_iter):
#     condition = True
#     n = 0
#     while condition:
#         n += 1
#         if n > max_iter:
#             return 'iter'
#         x2 = x0 - (x1-x0) * f(x0)/(f(x1) - f(x0))
#         if f(x0) * f(x2) < 0:
#             x1 = x2
#         else:
#             x0 = x2
#         condition = abs(f(x2)) > tol

#     return x2

In [2025]:
# Define the polynomial function
def polynomial(x, coeffs):
    return sum([coeff*(x**i) for i, coeff in enumerate(coeffs[::-1])])

In [2026]:
import time


def timed_blendBF(func, start, end):
    start_time = time.time()
    result = blendBF(func, start, end)
    end_time = time.time()
    return result, end_time - start_time

In [2027]:
df['root_hybrid'], df['hybrid_encode_time'] = zip(*df.apply(lambda row: timed_blendBF(lambda x: polynomial(x, row['polynomial_text_int']),
                                                                                      row['x'][0] -
                                                                                      range_extend,
                                                                                      row['x'][0]+range_extend), axis=1))

In [2028]:
# # Apply the blendBF function
# df['root_hybrid'] = df.apply(lambda row: blendBF(lambda x: polynomial(x, row['polynomial_text_int']),
#                                                  row['x'][0]-range_extend,
#                                                  row['x'][0]+range_extend), axis=1)

In [2029]:
# # Apply the bisection function
# df['root_bisection'] = df.apply(lambda row: bisection(lambda x: polynomial(x, row['polynomial_text_int']),
#   row['x'][0]-range_extend,
#   row['x'][0]+range_extend), axis=1)


# # Apply the false position function
# df['root_falsePosition'] = df.apply(lambda row: falsePosition(lambda x: polynomial(x, row['polynomial_text_int']),
#                                                               row['x'][0] -
#                                                               range_extend,
#                                                               row['x'][0]+range_extend), axis=1)

In [2030]:
length = len(df)
iter_rows = len(df[df['root_hybrid'] == 'iter'])
div_rows = len(df[df['root_hybrid'] == 'div'])
print(
    f"""
rows = {length}
iter = {iter_rows}
div = {div_rows}
rows - (iter + div) = {length - (iter_rows + div_rows)}
""")


rows = 487
iter = 147
div = 0
rows - (iter + div) = 340



In [2031]:
df = df.loc[df['root_hybrid'] != 'iter']

In [2032]:
df['text_from_root_hybrid'] = df.apply(lambda row: polynomial(
    row['root_hybrid'], row['polynomial']), axis=1)

In [2033]:
# df['text_from_root_bisection'] = df.apply(lambda row: polynomial(
#     row['root_bisection'], row['polynomial']), axis=1)


# df['text_from_root_FP'] = df.apply(lambda row: polynomial(
#     row['root_falsePosition'], row['polynomial']), axis=1)

In [2034]:
# text_from_root is the text representation from the root of
# hybrid algo
df['rounded_root_hybrid'] = df['text_from_root_hybrid'].round()

In [2035]:
# df['rounded_root_bisection'] = df['text_from_root_bisection'].round()
# df['rounded_root_FP'] = df['text_from_root_FP'].round()

In [2036]:
def intToTextUTF(num):
    start_time = time.time()
    try:
        # Convert the integer to bytes using UTF-8 encoding
        byte_representation = num.to_bytes(
            (num.bit_length() + 7) // 8, byteorder='big')
        # Convert the bytes to string using UTF-8 encoding
        text = byte_representation.decode('utf-8')
        end_time = time.time()
        return text, end_time - start_time
    except Exception:
        end_time = time.time()
        return "conv", end_time - start_time

In [2037]:
# df = df[df['hybrid_decode'] == df['rand_text']]

In [2038]:
df['hybrid_decode'], df['hybrid_decode_time'] = zip(
    *df['rounded_root_hybrid'].astype(int).apply(intToTextUTF))

In [2039]:
df['total_time_hybrid'] = df['hybrid_decode_time'] + df['hybrid_encode_time']

In [2040]:
df = df.loc[df['hybrid_decode'] == df['rand_text']]

In [2041]:
# df['bisection_decode'] = df['rounded_root_bisection'].astype(
#     int).apply(intToTextUTF)

In [2042]:
# df['FP_decode'] = df['rounded_root_FP'].astype(int).apply(intToTextUTF)

In [2043]:
df_filtered = df[['rand_text', 'text_int', 'hybrid_decode',
                  'hybrid_decode_time', 'hybrid_encode_time',
                  'total_time_hybrid']]

In [2044]:
df.to_csv('data.csv', index=False)

In [2045]:
df_filtered.to_csv('filtered_dataset.csv', index=False)